In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_ML_Kaggle\Features\FeaturesST"

## <span style="color:yellow"> **Preparamos los Sets de Training** </span> 

In [3]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_ML_Kaggle\Training Sets"

In [4]:
auc_trn = pd.read_csv( loc_ts + "\\auc_21_23.csv" ); 
auc_tst = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

## <span style="color:yellow"> **Genero los Sets con el primer auction** </span> 

In [5]:
auc_trn["date"] =  pd.to_datetime( auc_trn["date"] ); auc_trn["_st"] = auc_trn["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_trn["_st"] = auc_trn["_st"].dt.total_seconds();
auc_tst["date"] =  pd.to_datetime( auc_tst["date"] ); auc_tst["_st"] = auc_tst["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_tst["_st"] = auc_tst["_st"].dt.total_seconds(); 

In [6]:
auc_trn_frt = auc_trn.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_tst_frt = auc_tst.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [8]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_ML_Kaggle\Labels"

In [9]:
lb_auc_trn = pd.read_csv( loc_lb + "\\label_auc_24_26.csv" );
lb_auc_tst = pd.read_csv( loc_lb + "\\target.csv" ); 

In [10]:
rh_trn = lb_auc_trn[ ["ref_hash"] ]
rh_tst = lb_auc_tst[ ["ref_hash"] ]

tg_trn = lb_auc_trn[ ["24_26_st"] ]

## <span style="color:yellow"> **=================================================================================================** </span> 

## <span style="color:green"> **Hora de la realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 2

In [11]:
hora_auc_trn = rh_trn.copy()
hora_auc_tst = rh_tst.copy()

hora_auc_trn["hour"] = auc_trn_frt["date"].dt.hour; hora_auc_trn["hour"] = hora_auc_trn["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
hora_auc_tst["hour"] = auc_tst_frt["date"].dt.hour; hora_auc_tst["hour"] = hora_auc_tst["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

hora_auc_trn.to_csv( loc_ftr + "\\hora_auc_trn.csv", index = False )
hora_auc_tst.to_csv( loc_ftr + "\\hora_auc_tst.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [12]:
main_ahr_trn = rh_trn.copy()
main_ahr_tst = rh_tst.copy()

hour_mode_trn = auc_trn[["device_id"]]; hour_mode_trn["hour"] = pd.to_datetime( auc_trn["date"] ).dt.hour
hour_mode_tst = auc_tst[["device_id"]]; hour_mode_tst["hour"] = pd.to_datetime( auc_tst["date"] ).dt.hour

hour_mode_trn = hour_mode_trn.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_tst = hour_mode_tst.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_trn = hour_mode_trn.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_tst = hour_mode_tst.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

main_ahr_trn = main_ahr_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_tst = main_ahr_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ahr_trn["main_hour"] =  main_ahr_trn["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_tst["main_hour"] =  main_ahr_tst["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ahr_trn.to_csv( loc_ftr + "\\main_ahr_trn.csv", index = False )
main_ahr_tst.to_csv( loc_ftr + "\\main_ahr_tst.csv", index = False )

## <span style="color:green"> **Cantidad de subastas por dispositivo en la ventana previa a la conversion** </span> 

In [13]:
cant_auc_trn = rh_trn.copy()
cant_auc_tst = rh_tst.copy()

cant_auc_trn = cant_auc_trn.merge( auc_trn["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_tst = cant_auc_tst.merge( auc_tst["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_trn.fillna( 0, inplace = True )
cant_auc_tst.fillna( 0, inplace = True )

cant_auc_trn.to_csv( loc_ftr + "\\cant_auc_trn.csv", index = False )
cant_auc_tst.to_csv( loc_ftr + "\\cant_auc_tst.csv", index = False )

## <span style="color:green"> **Ref Type del dispositivo que realizo la subasta** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [14]:
ref_type_trn = rh_trn.copy()
ref_type_tst = rh_tst.copy()

rt_ohe_trn = pd.get_dummies( auc_trn_frt["ref_type_id"] )
rt_ohe_tst = pd.get_dummies( auc_tst_frt["ref_type_id"] )

ref_type_trn = pd.concat( [ref_type_trn, rt_ohe_trn], axis = 1 )
ref_type_tst = pd.concat( [ref_type_tst, rt_ohe_tst], axis = 1 )

ref_type_trn.to_csv( loc_ftr + "\\ref_type_trn.csv", index = False )
ref_type_tst.to_csv( loc_ftr + "\\ref_type_tst.csv", index = False )

## <span style="color:green"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo)

In [15]:
srce_auc_trn = rh_trn.copy()
srce_auc_tst = rh_tst.copy()

main_source_trn = auc_trn.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_tst = auc_tst.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_trn = main_source_trn.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_trn["cant_auc"] )
main_sourcetst3 = main_source_tst.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_tst["cant_auc"] )

srce_auc_trn = srce_auc_trn.merge( main_source_trn.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_trn.fillna( "no_font", inplace = True )
srce_auc_tst = srce_auc_tst.merge( main_source_tst.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_tst.fillna( "no_font", inplace = True )

srce_auc_trn["to_count"] = 1; srce_auc_trn["source_id"] = srce_auc_trn[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_trn); del(srce_auc_trn["to_count"])
srce_auc_tst["to_count"] = 1; srce_auc_tst["source_id"] = srce_auc_tst[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_tst); del(srce_auc_tst["to_count"])

srce_auc_trn.to_csv( loc_ftr + "\\srce_auc_trn.csv", index = False )
srce_auc_tst.to_csv( loc_ftr + "\\srce_auc_tst.csv", index = False )

## <span style="color:green"> **Ref Hash** </span> 


#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de apariciones en las subastas en la ventana anterior)

In [17]:
rh_encod_trn = rh_trn.copy()
rh_encod_tst = rh_tst.copy()

appces_trn_auc = auc_trn["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )
appces_tst_auc = auc_tst["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )

appces_trn_auc["ref_hash_mean"] = appces_trn_auc["appearances"] / len( auc_trn ); appces_trn_auc.drop( "appearances", axis = 1, inplace = True )
appces_tst_auc["ref_hash_mean"] = appces_tst_auc["appearances"] / len( auc_tst ); appces_tst_auc.drop( "appearances", axis = 1, inplace = True )

rh_encod_trn = rh_encod_trn.merge( appces_trn_auc, how = "left", on = "ref_hash" )
rh_encod_tst = rh_encod_tst.merge( appces_tst_auc, how = "left", on = "ref_hash" )

rh_encod_trn.to_csv( loc_ftr + "\\rh_encod_trn.csv", index = False )
rh_encod_tst.to_csv( loc_ftr + "\\rh_encod_tst.csv", index = False )

## <span style="color:green"> **Dia de la semana realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/6 ) para agregarle periodicidad. Por ejemplo 0 (lunes) tengan un valor similar a 6 (domingo)

In [18]:
sdia_auc_trn = rh_trn.copy()
sdia_auc_tst = rh_tst.copy()

sdia_auc_trn["hour"] = auc_trn_frt["date"].dt.hour; sdia_auc_trn["hour"] = sdia_auc_trn["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )
sdia_auc_tst["hour"] = auc_tst_frt["date"].dt.hour; sdia_auc_tst["hour"] = sdia_auc_tst["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )

sdia_auc_trn.to_csv( loc_ftr + "\\sdia_auc_trn.csv", index = False )
sdia_auc_tst.to_csv( loc_ftr + "\\sdia_auc_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la primer auction en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer auction

In [19]:
frst_auc_trn = rh_trn.copy()
frst_auc_tst = rh_tst.copy()

first_auctions_rh_ins_trn = auc_trn[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auctions_rh_ins_tst = auc_tst[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auctions_rh_ins_trn["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auctions_rh_ins_tst["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auctions_rh_ins_trn.drop( "date", axis = 1, inplace = True )
first_auctions_rh_ins_tst.drop( "date", axis = 1, inplace = True )

frst_auc_trn = frst_auc_trn.merge( first_auctions_rh_ins_trn, how = "left", on = "ref_hash" )
frst_auc_tst = frst_auc_tst.merge( first_auctions_rh_ins_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_auc_trn.fillna( 3 * 24 * 3600, inplace = True )
frst_auc_tst.fillna( 3 * 24 * 3600, inplace = True )

frst_auc_trn.to_csv( loc_ftr + "\\frst_auc_trn.csv", index = False )
frst_auc_tst.to_csv( loc_ftr + "\\frst_auc_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultima auction en esa ventana** </span> 

In [21]:
last_auc_trn = rh_trn.copy()
last_auc_tst = rh_tst.copy()

last_auctions_rh_auc_trn = auc_trn[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
last_auctions_rh_auc_tst = auc_tst[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

last_auctions_rh_auc_trn["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_auc_trn["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_auctions_rh_auc_tst["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_auc_tst["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_auctions_rh_auc_trn.drop( "date", axis = 1, inplace = True )
last_auctions_rh_auc_tst.drop( "date", axis = 1, inplace = True )

last_auc_trn = last_auc_trn.merge( last_auctions_rh_auc_trn, how = "left", on = "ref_hash" )
last_auc_tst = last_auc_tst.merge( last_auctions_rh_auc_tst, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_auc_trn.fillna( 3 * 24 * 3600, inplace = True )
last_auc_tst.fillna( 3 * 24 * 3600, inplace = True )

last_auc_trn.to_csv( loc_ftr + "\\last_auc_trn.csv", index = False )
last_auc_tst.to_csv( loc_ftr + "\\last_auc_tst.csv", index = False )

## <span style="color:green"> **Participo en mas de una subasta** </span> 

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [23]:
cant_auc_trn = rh_trn.copy()
cant_auc_tst = rh_tst.copy()

cant_auc_trn = cant_auc_trn.merge( auc_trn["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_tst = cant_auc_tst.merge( auc_tst["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_trn.fillna( 0, inplace = True )
cant_auc_tst.fillna( 0, inplace = True )

cant_auc_trn['mt_1_auc'] = (cant_auc_trn['mt_1_auc'] > 1).astype('int8')
cant_auc_tst['mt_1_auc'] = (cant_auc_tst['mt_1_auc'] > 1).astype('int8')

cant_auc_trn.to_csv( loc_ftr + "\\m1oh_auc_trn.csv", index = False )
cant_auc_tst.to_csv( loc_ftr + "\\m1oh_auc_tst.csv", index = False )

## <span style="color:green"> **Recibio subasta entre 21 y 3** </span> 

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [25]:
auc_trn['auc_21_3'] = (auc_trn['date'].dt.hour < 4) | (auc_trn['date'].dt.hour > 20)
auc_tst['auc_21_3'] = (auc_tst['date'].dt.hour < 4) | (auc_tst['date'].dt.hour > 20)

auc_night_trn = rh_trn.copy()
auc_night_tst = rh_tst.copy()

hour_mode_trn = auc_trn.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_tst = auc_tst.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_trn = auc_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
auc_night_tst = auc_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

auc_night_trn["auc_21_3"] =  (auc_night_trn["auc_21_3"] > 1).astype('int8')
auc_night_tst["auc_21_3"] =  (auc_night_tst["auc_21_3"] > 1).astype('int8')

auc_night_trn.to_csv( loc_ftr + "\\auc_nght_trn.csv", index = False )
auc_night_tst.to_csv( loc_ftr + "\\auc_nght_tst.csv", index = False )

del auc_trn['auc_21_3']
del auc_tst['auc_21_3']

## <span style="color:green"> **Recibio subasta entre 4 y 10** </span> 

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [27]:
auc_trn['auc_4_10'] = (auc_trn['date'].dt.hour > 3) & (auc_trn['date'].dt.hour < 11) 
auc_tst['auc_4_10'] = (auc_tst['date'].dt.hour > 3) & (auc_tst['date'].dt.hour < 11)

auc_morn_trn = rh_trn.copy()
auc_morn_tst = rh_tst.copy()

hour_mode_trn = auc_trn.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_tst = auc_tst.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_trn = auc_morn_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
auc_morn_tst = auc_morn_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

auc_morn_trn["auc_4_10"] =  (auc_morn_trn["auc_4_10"] > 1).astype('int8')
auc_morn_tst["auc_4_10"] =  (auc_morn_tst["auc_4_10"] > 1).astype('int8')

auc_morn_trn.to_csv( loc_ftr + "\\auc_morn_trn.csv", index = False )
auc_morn_tst.to_csv( loc_ftr + "\\auc_morn_tst.csv", index = False )

del auc_trn['auc_4_10']
del auc_tst['auc_4_10']

## <span style="color:green"> **Recibio subasta entre 11 y 15** </span> 

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [29]:
auc_trn['auc_11_15'] = (auc_trn['date'].dt.hour > 10) & (auc_trn['date'].dt.hour < 16) 
auc_tst['auc_11_15'] = (auc_tst['date'].dt.hour > 10) & (auc_tst['date'].dt.hour < 16)

auc_midday_trn = rh_trn.copy()
auc_midday_tst = rh_tst.copy()

hour_mode_trn = auc_trn.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_tst = auc_tst.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_trn = auc_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
auc_midday_tst = auc_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

auc_midday_trn["auc_11_15"] =  (auc_midday_trn["auc_11_15"] > 1).astype('int8')
auc_midday_tst["auc_11_15"] =  (auc_midday_tst["auc_11_15"] > 1).astype('int8')

auc_midday_trn.to_csv( loc_ftr + "\\auc_mday_trn.csv", index = False )
auc_midday_tst.to_csv( loc_ftr + "\\auc_mday_tst.csv", index = False )

del auc_trn['auc_11_15']
del auc_tst['auc_11_15']

## <span style="color:green"> **Recibio subasta entre 16 y 20** </span> 

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [ ]:
auc_trn['auc_16_20'] = (auc_trn['date'].dt.hour > 15) & (auc_trn['date'].dt.hour < 21) 
auc_tst['auc_16_20'] = (auc_tst['date'].dt.hour > 15) & (auc_tst['date'].dt.hour < 21)

auc_after_trn = rh_trn.copy()
auc_after_tst = rh_tst.copy()

hour_mode_trn = auc_trn.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_tst = auc_tst.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_trn = auc_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
auc_after_tst = auc_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

auc_after_trn["auc_16_20"] =  (auc_after_trn["auc_16_20"] > 1).astype('int8')
auc_after_tst["auc_16_20"] =  (auc_after_tst["auc_16_20"] > 1).astype('int8')

auc_after_trn.to_csv( loc_ftr + "\\auc_aftr_trn.csv", index = False )
auc_after_tst.to_csv( loc_ftr + "\\auc_aftr_tst.csv", index = False )

del auc_trn['auc_16_20']
del auc_tst['auc_16_20']